In [1]:
#Import packages
import pandas as pd
import numpy as np
import re
import warnings
import math
warnings.filterwarnings('ignore')

## HF-ACTION

In [2]:
#Note - only looking at
demog = pd.read_sas("../Data Validation/HF-ACTION/Raw Data/demog.sas7bdat", index='newid').sort_index()
analysis = pd.read_sas("../Data Validation/HF-ACTION/Raw Data/analysis.sas7bdat", index='newid').sort_index()
medhx1 = pd.read_sas("../Data Validation/HF-ACTION/Raw Data/medhx1.sas7bdat", index='newid').sort_index()
chfevent = pd.read_sas("../Data Validation/HF-ACTION/Raw Data/chfevent.sas7bdat", index='newid').sort_index()
cmed = pd.read_sas("../Data Validation/HF-ACTION/Raw Data/cmed.sas7bdat", index='newid').sort_index()
cechosp = pd.read_sas("../Data Validation/HF-ACTION/Raw Data/cechosp.sas7bdat", index='newid').sort_index()
saarrh = pd.read_sas("../Data Validation/HF-ACTION/Raw Data/saarrh.sas7bdat", index='newid').sort_index()
medhx2 = pd.read_sas("../Data Validation/HF-ACTION/Raw Data/medhx2.sas7bdat", index='newid').sort_index()
hospital = pd.read_sas("../Data Validation/HF-ACTION/Raw Data/hospital.sas7bdat", index='newid').sort_index()
othevent = pd.read_sas("../Data Validation/HF-ACTION/Raw Data/othevent.sas7bdat", index='newid').sort_index()


set(demog.index) == set(analysis.index)
np.setdiff1d(analysis.index, demog.index)

array([], dtype=object)

In [3]:
cohort = pd.DataFrame()
cohort['ID'] = demog.index
cohort = cohort.set_index('ID')

cohort['Age'] = demog['age'].tolist()

lst = []
for idx in demog.index:
    pt = demog.loc[idx]
    if pt['GENDER'] == 1.0:
        lst.append(1)
    else:
        lst.append(2)
cohort['Gender'] = lst


lst = []
for idx in analysis.index:
    pt = analysis.loc[idx]
    if pt['racec'] == 2:
        lst.append(1)
    else:
        lst.append(2)
cohort['Race'] = lst

#weight, convert to all be Kg
cohort['Wt'] = analysis['weightkg'].tolist()

#bmi
cohort['BMI'] = analysis['bmi'].tolist()

#ejf
cohort['EjF'] = analysis['bipllvef'].tolist()

#NYHA
cohort['NYHA'] = analysis['nyhacl'].tolist()

cohort.head(5)

,Age,Gender,Race,Wt,BMI,EjF,NYHA
ID,,,,,,,
b'HFACT00001',62.0,1,1,78.0,25.18,32.24,2.0
b'HFACT00002',75.0,1,2,68.0,22.96,21.71,2.0
b'HFACT00003',69.0,1,1,80.0,24.14,20.16,2.0
b'HFACT00004',45.0,1,1,114.0,32.35,23.79,2.0
b'HFACT00005',75.0,1,1,106.0,30.87,NaN,2.0


In [4]:
cohort['Gender'].value_counts()

1    1531
2     599
Name: Gender, dtype: int64

In [5]:
#conditions, Ischemic
lst = []
for idx in analysis.index:
    pt = analysis.loc[idx]
    if pt['etiology']== 1:
        lst.append(1)
    else:
        lst.append(0)
cohort['ISCH'] = lst

#Nonischemic
lst = []
for idx in analysis.index:
    pt = analysis.loc[idx]
    if pt['etiology'] == 2:
        lst.append(1)
    else:
        lst.append(0)
cohort['NonISCH'] = lst

#AF - says yes/no in description, assumed 1 is yes and 0 is no - afib and flutter are recorded together.
lst = []
for idx in analysis.index:
    pt = analysis.loc[idx]
    if pt['afibflut'] == 1:
        lst.append(1)
    else:
        lst.append(0)
cohort['AF'] = lst

#Angina
lst = []
for idx in analysis.index:
    pt = analysis.loc[idx]
    if pt['angina'] == 1:
        lst.append(1)
    else:
        lst.append(0)
cohort['ANGP'] = lst

#ARRH
lst = []
for idx in analysis.index:
    pt = medhx1.loc[idx]
    if type(pt) != pd.Series:
        try:
            pt = pt[pt['FORM'] == b'BASELINE']
            nh = pt['ARR'].item()
        except:
            nh = None
    else:
        nh = pt['ARR']
    lst.append(nh)
cohort['ARRH'] = lst

# CARREST
lst = []
for idx in analysis.index: # is this working as expected?
    if idx not in saarrh.index:
        lst.append(np.nan)
    else:
        pt = saarrh.loc[idx]
        car = pt['CARREST'].tolist()
        if type(car) != list:
            car = [car]
        if 1.0 in car:
            lst.append(1)
        else:
            lst.append(0)
cohort['CARREST'] = lst

#COPD
lst = []
for idx in analysis.index:
    pt = analysis.loc[idx]
    if pt['COPD'] == 1:
        lst.append(1)
    else:
        lst.append(0)
cohort['COPD'] = lst

#Depr
lst = []
for idx in analysis.index:
    pt = analysis.loc[idx]
    if pt['depress'] == 1:
        lst.append(1)
    else:
        lst.append(0)
cohort['DEPR'] = lst

#Diab
lst = []
for idx in analysis.index:
    pt = analysis.loc[idx]
    if pt['diabetes'] == 1:
        lst.append(1)
    else:
        lst.append(0)
cohort['DIAB'] = lst

# HEPT
lst = []
for idx in analysis.index: # is this working as expected?
    if idx not in chfevent.index:
        lst.append(np.nan)
    else:
        pt = chfevent.loc[idx]
        car = pt['WAHH'].tolist()
        if type(car) != list:
            car = [car]
        if 1.0 in car:
            lst.append(1)
        else:
            lst.append(0)
cohort['HEPT'] = lst

#HTN 
lst = []
for idx in analysis.index:
    pt = analysis.loc[idx]
    medPT = medhx1.loc[idx]
    if pt['hyperten'] == 1 or medPT['HYPTN'] == 1:
        lst.append(1)
    else:
        lst.append(0)
cohort['HTN'] = lst

#Malig -- medhx2 - cancer hx in last 5 years
lst = []
for idx in medhx2.index:
    pt = medhx2.loc[idx]
    if pt['CANCER'] == 1:
        lst.append(1)
    else:
        lst.append(0)
cohort['MALIG'] = lst

#Renal
lst = []
for idx in analysis.index:
    pt = analysis.loc[idx]
    if pt['renaldys'] == 1:
        lst.append(1)
    else:
        lst.append(0)
cohort['RENAL'] = lst

#Smok
lst = []
for idx in analysis.index:
    pt = analysis.loc[idx]
    if pt['smokenever'] == 1:
        lst.append(0)
    else:
        lst.append(1)
cohort['SMOKING'] = lst

#Stroke TIA --- othevent TIA?
lst = []
for idx in analysis.index:
    pt = analysis.loc[idx]
    medPt = medhx2.loc[idx]
    oStroke = []
    oTIA = []
    if idx in othevent.index:
        othPt = othevent.loc[idx]
        oStroke = othPt['STROKE'].tolist()
        if type(oStroke) != list:
            oStroke = [oStroke]
        oTIA = othPt['TIA'].tolist()
        if type(oTIA) != list:
            oTIA = [oTIA]
    
    if pt['stroke'] == 1 or medPt['STRK'] == 1 or 1.0 in oStroke or 1.0 in oTIA:
        lst.append(1)
    else:
        lst.append(0)
cohort['StrokeTIA'] = lst

#VAHD -- medhx1 - CONGHF, medhx2 - PVD
lst = []
for idx in analysis.index:
    pt = analysis.loc[idx]
    medPt = medhx2.loc[idx]
    if pt['PVD'] == 1 or medPt['PVD'] == 1:
        lst.append(1)
    else:
        lst.append(0)
cohort['VAHD'] = lst

# VF
lst = []
for idx in analysis.index: # is this working as expected?
    if idx not in saarrh.index:
        lst.append(np.nan)
    else:
        pt = saarrh.loc[idx]
        car = pt['VENTFIB'].tolist()
        if type(car) != list:
            car = [car]
        if 1.0 in car:
            lst.append(1)
        else:
            lst.append(0)
cohort['VF'] = lst

#VHD - medhx1 - CARDPR2 - found nothing on aorta, tricuspid or stenosis
lst = []
for idx in analysis.index:
    pt = analysis.loc[idx]
    if pt['valvsurg'] == 1 or np.isnan(pt['mitregrg']) == False :
        lst.append(1)
    else:
        lst.append(0)
cohort['VHD'] = lst

#VT
lst = []
for idx in analysis.index:
    pt = medhx1.loc[idx]
    tach1 = []
    tach2 = []
    if idx in saarrh.index:
        othPt = saarrh.loc[idx]
        tach1 = othPt['VENTTACH'].tolist()
        if type(tach1) != list:
            tach1 = [tach1]
        tach2 = othPt['SUPTACH'].tolist()
        if type(tach2) != list:
            tach2 = [tach2]
    
    if pt['VTACH'] == 1 or 1.0 in tach1 or 1.0 in tach2:
        lst.append(1)
    else:
        lst.append(0)
cohort['VT'] = lst

#CABG - medhx = CARDPR1
lst = []
for idx in analysis.index:
    pt = analysis.loc[idx]
    medPt = medhx1.loc[idx]
    if pt['cabg'] == 1 or medPt['CARDPR1'] == 1:
        lst.append(1)
    else:
        lst.append(0)
cohort['CABG'] = lst

#ICD
lst = []
for idx in analysis.index:
    pt = analysis.loc[idx]
    medPt = medhx1.loc[idx]
    if pt['aicd'] == 1 or medPt['CARDPR5'] == 1:
        lst.append(1)
    else:
        lst.append(0)
cohort['ICD'] = lst

#PACE
lst = []
for idx in analysis.index:
    pt = analysis.loc[idx]
    medPt = medhx1.loc[idx]
    if pt['bivpacer'] == 1 or pt['pacer'] == 1 or medPt['CARDPR4'] == 1 or medPt['CARDPR6'] == 1:
        lst.append(1)
    else:
        lst.append(0)
cohort['PACE'] = lst

#PTCI
lst = []
for idx in analysis.index:
    pt = analysis.loc[idx]
    if pt['priorpci'] == 1:
        lst.append(1)
    else:
        lst.append(0)
cohort['PTCI'] = lst

cohort

,Age,Gender,Race,Wt,BMI,EjF,NYHA,ISCH,NonISCH,AF,...,SMOKING,StrokeTIA,VAHD,VF,VHD,VT,CABG,ICD,PACE,PTCI
ID,,,,,,,,,,,,,,,,,,,,,
b'HFACT00001',62.0,1,1,78.0,25.18,32.24,2.0,0,1,0,...,1,0,0,NaN,0,1,0,1,0,0
b'HFACT00002',75.0,1,2,68.0,22.96,21.71,2.0,0,1,1,...,1,1,0,NaN,1,0,0,0,0,0
b'HFACT00003',69.0,1,1,80.0,24.14,20.16,2.0,1,0,0,...,1,0,0,NaN,1,0,1,1,1,0
b'HFACT00004',45.0,1,1,114.0,32.35,23.79,2.0,1,0,0,...,1,0,0,NaN,1,1,0,1,0,1
b'HFACT00005',75.0,1,1,106.0,30.87,NaN,2.0,1,0,0,...,1,0,0,NaN,1,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b'HFACT02135',60.0,1,1,95.0,32.89,NaN,3.0,1,0,0,...,1,0,1,NaN,1,0,1,0,0,0
b'HFACT02136',63.0,1,1,97.0,27.60,17.32,3.0,1,0,1,...,1,1,0,NaN,1,0,0,1,0,0
b'HFACT02137',62.0,1,2,91.0,26.52,28.58,2.0,1,0,1,...,1,0,0,0.0,1,1,0,1,0,0


In [6]:
#Exercise

#walk distance in feet
#convert meters --> feet
lst = []
for idx in analysis.index:
    pt = analysis.loc[idx]
    ex = pt['sixmwlkd'] * 3.28
    lst.append(ex)
cohort['SixFtWlk'] = lst

cohort['VO2'] = analysis['peakvo2'].tolist()

#LABS
cohort['BUN'] = analysis['bun'].tolist()
cohort['CRT'] = analysis['creatnin'].tolist()

#MEDS
#ACE
lst = []
for idx in analysis.index:
    pt = cmed.loc[idx]
    if type(pt) != pd.Series:
        pt = pt[pt['FORM'] == b'BASELINE']
        nh = pt['ACE'].item()
    else:
        if pt['FORM'] == b'BASELINE':
            nh = pt['ACE']
        else:
            nh = np.nan
    lst.append(nh)
cohort['ACE'] = lst

#BETA
lst = []
for idx in analysis.index:
    pt = cmed.loc[idx]
    if type(pt) != pd.Series:
        pt = pt[pt['FORM'] == b'BASELINE']
        nh = pt['BETA'].item()
    else:
        if pt['FORM'] == b'BASELINE':
            nh = pt['BETA']
        else:
            nh = np.nan
    lst.append(nh)
cohort['BET'] = lst

#NIT
cohort['NIT'] = analysis['nitrate'].tolist()

#DIUR
lst = [] 
for idx in analysis.index:
    pt = analysis.loc[idx]
    if pt['loopdiur'] == 1 or pt['nonloop'] == 1 or pt['othdiur'] == 1 or pt['bothdiur'] == 1:
        lst.append(1.0)
    else:
        lst.append(0.0)
cohort['DIUR'] = lst

cohort

,Age,Gender,Race,Wt,BMI,EjF,NYHA,ISCH,NonISCH,AF,...,PACE,PTCI,SixFtWlk,VO2,BUN,CRT,ACE,BET,NIT,DIUR
ID,,,,,,,,,,,,,,,,,,,,,
b'HFACT00001',62.0,1,1,78.0,25.18,32.24,2.0,0,1,0,...,0,0,1377.600000,10.4,28.067227,1.752036,0.0,1.0,0.0,1.0
b'HFACT00002',75.0,1,2,68.0,22.96,21.71,2.0,0,1,1,...,0,0,602.845632,8.4,29.000000,1.200000,0.0,1.0,0.0,1.0
b'HFACT00003',69.0,1,1,80.0,24.14,20.16,2.0,1,0,0,...,1,0,1282.171680,22.3,13.000000,1.100000,1.0,1.0,0.0,1.0
b'HFACT00004',45.0,1,1,114.0,32.35,23.79,2.0,1,0,0,...,0,1,1779.544320,21.9,8.000000,0.900000,1.0,1.0,0.0,1.0
b'HFACT00005',75.0,1,1,106.0,30.87,NaN,2.0,1,0,0,...,1,1,1338.240000,16.7,23.000000,1.300000,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b'HFACT02135',60.0,1,1,95.0,32.89,NaN,3.0,1,0,0,...,0,0,299.923200,12.2,NaN,NaN,0.0,1.0,0.0,0.0
b'HFACT02136',63.0,1,1,97.0,27.60,17.32,3.0,1,0,1,...,0,0,1056.729408,13.4,18.000000,1.200000,1.0,1.0,0.0,1.0
b'HFACT02137',62.0,1,2,91.0,26.52,28.58,2.0,1,0,1,...,0,0,989.746560,8.8,29.000000,1.400000,1.0,1.0,1.0,1.0


In [7]:
#Hemo
cohort['HRTRT'] = analysis['basehr'].tolist()
cohort['BPSYS'] = analysis['sbp'].tolist()
cohort['BPDIAS'] = analysis['dbp'].tolist()

idx = sorted(set(cohort.index))

#MAP
base = []
for i in idx:
    t = cohort.loc[i]
    b = t['BPSYS'] + ((2 * t['BPDIAS']) / 3)
    base.append(b)

cohort['MAP'] = base

#PP
base = []
for i in idx:
    t = cohort.loc[i]
    b = t['BPSYS'] - t['BPDIAS']
    base.append(b)

cohort['PP'] = base

#PPP - found no discharge
base = []
for i in idx:
    t = cohort.loc[i]
    b = t['PP'] / t['BPSYS']
    base.append(b)

cohort['PPP'] = base

#PPRatio -- no discharge pp
base = []
for i in idx:
    t = cohort.loc[i]
    b = t['PP'] / t['HRTRT']
    base.append(b)
    
cohort['PPRatio'] = base

cohort.head(5)

,Age,Gender,Race,Wt,BMI,EjF,NYHA,ISCH,NonISCH,AF,...,BET,NIT,DIUR,HRTRT,BPSYS,BPDIAS,MAP,PP,PPP,PPRatio
ID,,,,,,,,,,,,,,,,,,,,,
b'HFACT00001',62.0,1,1,78.0,25.18,32.24,2.0,0,1,0,...,1.0,0.0,1.0,70.0,100.0,70.0,146.666667,30.0,0.300000,0.428571
b'HFACT00002',75.0,1,2,68.0,22.96,21.71,2.0,0,1,1,...,1.0,0.0,1.0,120.0,102.0,72.0,150.000000,30.0,0.294118,0.250000
b'HFACT00003',69.0,1,1,80.0,24.14,20.16,2.0,1,0,0,...,1.0,0.0,1.0,82.0,122.0,82.0,176.666667,40.0,0.327869,0.487805
b'HFACT00004',45.0,1,1,114.0,32.35,23.79,2.0,1,0,0,...,1.0,0.0,1.0,74.0,108.0,80.0,161.333333,28.0,0.259259,0.378378
b'HFACT00005',75.0,1,1,106.0,30.87,NaN,2.0,1,0,0,...,1.0,0.0,1.0,63.0,106.0,68.0,151.333333,38.0,0.358491,0.603175


In [8]:
#add additional variables
#bnp
lst = [] 
for idx in analysis.index:
    pt = analysis.loc[idx]
    lst.append(pt['bnp'])

cohort['BNP'] = lst

#diur dose
lst = [] 
for idx in analysis.index:
    pt = analysis.loc[idx]
    lst.append(pt['loopdose'])

cohort['DIURDSE'] = lst
cohort['FurosemideDse'] = lst

#total chol
lst = [] 
for idx in analysis.index:
    pt = analysis.loc[idx]
    lst.append(pt['totcholc'])

cohort['TOTChol'] = lst

cohort

,Age,Gender,Race,Wt,BMI,EjF,NYHA,ISCH,NonISCH,AF,...,BPSYS,BPDIAS,MAP,PP,PPP,PPRatio,BNP,DIURDSE,FurosemideDse,TOTChol
ID,,,,,,,,,,,,,,,,,,,,,
b'HFACT00001',62.0,1,1,78.0,25.18,32.24,2.0,0,1,0,...,100.0,70.0,146.666667,30.0,0.300000,0.428571,NaN,40.0,40.0,177.882
b'HFACT00002',75.0,1,2,68.0,22.96,21.71,2.0,0,1,1,...,102.0,72.0,150.000000,30.0,0.294118,0.250000,172.0,40.0,40.0,219.000
b'HFACT00003',69.0,1,1,80.0,24.14,20.16,2.0,1,0,0,...,122.0,82.0,176.666667,40.0,0.327869,0.487805,100.0,20.0,20.0,206.000
b'HFACT00004',45.0,1,1,114.0,32.35,23.79,2.0,1,0,0,...,108.0,80.0,161.333333,28.0,0.259259,0.378378,NaN,40.0,40.0,164.000
b'HFACT00005',75.0,1,1,106.0,30.87,NaN,2.0,1,0,0,...,106.0,68.0,151.333333,38.0,0.358491,0.603175,NaN,40.0,40.0,177.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b'HFACT02135',60.0,1,1,95.0,32.89,NaN,3.0,1,0,0,...,140.0,70.0,186.666667,70.0,0.500000,0.752688,NaN,0.0,0.0,NaN
b'HFACT02136',63.0,1,1,97.0,27.60,17.32,3.0,1,0,1,...,90.0,60.0,130.000000,30.0,0.333333,0.454545,955.0,40.0,40.0,144.000
b'HFACT02137',62.0,1,2,91.0,26.52,28.58,2.0,1,0,1,...,130.0,88.0,188.666667,42.0,0.323077,0.560000,NaN,80.0,80.0,105.000


In [9]:
#Make separate datasets
#No hemo data so no hemo dataset

#All Data - conditions
varz = ['Age', 'Gender', 'Race', 'Wt', 'BMI', 'InitialHospDays',
       'TotalHospDays', 'NYHA', 'MLHFS', 'AF', 'AlchE', 'ANGP', 'ARRH',
       'CARREST', 'CVD', 'COPD', 'DEPR', 'DIAB', 'GOUT', 'HEPT', 'HTN',
       'MALIG', 'RENAL', 'SMOKING', 'STERD', 'StrokeTIA', 'VAHD', 'VF', 'VHD',
       'VT', 'ISCH', 'NonISCH', 'CABG', 'HTRANS', 'ICD', 'PACE', 'PTCI',
       'SixFtWlk', 'VO2', 'ALB', 'ALT', 'AST', 'BUN', 'CRT', 'DIAL', 'HEC',
       'HEM', 'PLA', 'POT', 'SOD', 'TALB', 'TOTP', 'WBC', 'ACE', 'BET', 'NIT',
       'DIUR', 'DIURDSE', 'FurosemideDse', 'Bumetanide', 'Torsemide', 'INOT',
       'EjF', 'BPDIAS', 'BPSYS', 'HR', 'RAP', 'PAS', 'PAD', 'PAMN', 'PCWP',
       'CO', 'CI', 'SVR', 'MIXED', 'HRTRT', 'RAT', 'MAP', 'MPAP', 'CPI', 'PP',
       'PPP', 'PAPP', 'PPRatio', 'PAPi', 'SAPi', 'CPP', 'PRAPRat', 'BNP',
       'CPR', 'MEVT', 'HGB', 'URIC', 'TOTChol']


giantList = []
for i in range(len(cohort)):
    df = cohort.iloc[i]
    row = []
    row.append(df.name)
    for col in varz:
        if col in cohort.columns: #value for both
            row.append(df[col])
        else:
            row.append(0)

    giantList.append(row)

varz.insert(0, 'ID')
allData = pd.DataFrame(giantList, columns=varz).set_index('ID', drop=True)

allData

,Age,Gender,Race,Wt,BMI,InitialHospDays,TotalHospDays,NYHA,MLHFS,AF,...,PAPi,SAPi,CPP,PRAPRat,BNP,CPR,MEVT,HGB,URIC,TOTChol
ID,,,,,,,,,,,,,,,,,,,,,
b'HFACT00001',62.0,1.0,1.0,78.0,25.18,0,0,2.0,0,0.0,...,0,0,0,0,NaN,0,0,0,0,177.882
b'HFACT00002',75.0,1.0,2.0,68.0,22.96,0,0,2.0,0,1.0,...,0,0,0,0,172.0,0,0,0,0,219.000
b'HFACT00003',69.0,1.0,1.0,80.0,24.14,0,0,2.0,0,0.0,...,0,0,0,0,100.0,0,0,0,0,206.000
b'HFACT00004',45.0,1.0,1.0,114.0,32.35,0,0,2.0,0,0.0,...,0,0,0,0,NaN,0,0,0,0,164.000
b'HFACT00005',75.0,1.0,1.0,106.0,30.87,0,0,2.0,0,0.0,...,0,0,0,0,NaN,0,0,0,0,177.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b'HFACT02135',60.0,1.0,1.0,95.0,32.89,0,0,3.0,0,0.0,...,0,0,0,0,NaN,0,0,0,0,NaN
b'HFACT02136',63.0,1.0,1.0,97.0,27.60,0,0,3.0,0,1.0,...,0,0,0,0,955.0,0,0,0,0,144.000
b'HFACT02137',62.0,1.0,2.0,91.0,26.52,0,0,2.0,0,1.0,...,0,0,0,0,NaN,0,0,0,0,105.000


In [10]:
#save to file
allData.to_csv("Preprocessed Data/AllDataHF-ACTION.csv")


In [11]:
#Make Labels
labels = pd.DataFrame()
labels['ID'] = analysis.index
labels = labels.set_index('ID')

dthList =  []
rhspList = []
readmList = []
idx = sorted(set(analysis.index))
for i in idx:
    t = analysis.loc[i]

    #Death outcome
    dth = t["death"] #death is death - all causes, no LVAD or Transplant found in data set
    if dth == 1:
        dthList.append(1)
    else:
        dthList.append(0)
    #rehosp
    rhsp = t['hospsxmo']
    if rhsp >= 2:
        rhspList.append(1)
    else:
        rhspList.append(0)
    
    #readm
    if i in cechosp.index:
        c = cechosp.loc[i]
        readm = c['hspdays']
#         print(readm)
        if type(readm) != pd.Series:
            if readm < 30:
                readmList.append(1)
#                 print("ADDING READM\n")
            else:
                readmList.append(0)
            
        else:
            readm = readm.tolist()
            truval = False
            for elem in readm:
                if elem < 30:
                    truval = True
                    
            if truval:
                readmList.append(1)
#                 print("ADDING READM\n")
            else:
                readmList.append(0)

    else:
        readmList.append(0)

labels['Death'] = dthList
labels['Rehosp'] = rhspList
labels['Readmission'] = readmList

labels

,Death,Rehosp,Readmission
ID,,,
b'HFACT00001',0,0,0
b'HFACT00002',0,1,1
b'HFACT00003',0,0,0
b'HFACT00004',0,0,0
b'HFACT00005',0,0,0
...,...,...,...
b'HFACT02135',0,1,0
b'HFACT02136',1,0,0
b'HFACT02137',1,0,0


In [ ]:
labels.to_csv("Preprocessed Data/LabelsHF-ACTION.csv")


In [12]:
labels.to_csv("Preprocessed Data/LabelsHF-ACTION.csv")
